In [1]:
#pip install pyLDAvis #install pyLDAvis


In [2]:
#pip install -U spacy

In [3]:
#basic libraries
import pandas as pd
import numpy as np
import re, pprint

#Dimension Reduction Libraries
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#Visualizations
import pyLDAvis
import pyLDAvis.sklearn

#NLP Libraries
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
import spacy


/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/anaconda3/lib/python3.7/site-packages/thinc/neural/train.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from .optimizers import Adam, linear_decay
/anaconda3/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, Size

# 1. Reading in CSV and Dataframe Cleaning

In [4]:
#Data already taken from API, will run additional requests with different filtering parameters as needed
#designate paths
path1 = 'api_data.csv'
path2 = 'api_data_btc_eth_filter.csv'
path3 = 'api_data_btc_eth_regulations_moreetc_regs_filter_yahoofinance.csv'
path4 = 'api_data_btc_eth_regulations_moreetc_regs_filter.csv'
path5 = 'api_data_no_filters.csv'

#df=pd.concat([pd.read_csv(path1),pd.read_csv(path2),pd.read_csv(path3),pd.read_csv(path4), pd.read_csv(path5)])

df=pd.concat([pd.read_csv(path2),pd.read_csv(path3),pd.read_csv(path4)])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [5]:
df.id = df.id.astype(str)#changing ID to str

In [6]:
dropcol = ['Unnamed: 0','downvotes','guid','imageurl','lang','source_info','upvotes','url']

df_analysis = df.drop(dropcol,axis=1)

In [7]:
#remove duplicate articles

df_analysis.drop_duplicates('id',inplace = True)

In [8]:
df_analysis = df_analysis.reset_index()
df_analysis = df_analysis.drop(['index'],axis = 1)

In [9]:
df_analysis["text"] = df_analysis["title"].map(str) + ' '+ df_analysis["body"].map(str)

In [10]:
#need to do some regrex cleaning on text column
#Regex to get rid of numbers so that only ingredients are left

#use an apply function to apply regrex to each row in nlp_ingredients
df_analysis['text'] = df_analysis['text'].apply(lambda x: re.sub(r'\b\d+\b', '', x))

#remove punctuation
df_analysis['text'] = df_analysis['text'].apply(lambda x: re.sub(r'[^\w\s]','',x))

#make all ingredients lowercase
df_analysis['text'] = df_analysis['text'].apply(lambda x: x.lower())



In [11]:
#Create a year_month column
df_analysis['year_month'] = df_analysis['date_time'].str[0:7]
#Create a year_month_day column
df_analysis['year_month_day'] = df_analysis['date_time'].str[0:10]


In [12]:
#convert data_time back to date_time object
df_analysis['date_time'] = pd.to_datetime(df_analysis['date_time'])

In [13]:
#Create year_week column
df_analysis['date_time_week'] = df_analysis.date_time.dt.strftime("%Y-%W")

In [14]:
df_analysis.head()

,body,categories,date_time,id,published_on,source,tags,title,text,year_month,year_month_day,date_time_week
0,Bitcoin has been trading for a premium of betw...,BTC|Market|Trading,2019-08-15 22:00:47,2813980,1565906447,newsbtc,Bitcoin|Opinion|argentina|bitcoin|safe haven,Argentine Bitcoin Premium Shows Relativity of ...,argentine bitcoin premium shows relativity of ...,2019-08,2019-08-15,2019-32
1,Bitcoin’s market valuation has already made so...,BTC|Trading|Market,2019-08-15 22:00:21,2813979,1565906421,bitcoinist,Bitcoin|Bitcoin Price|News|News teaser|bitcoin...,Brandt’s Momentum Indicator Highlights Potenti...,brandts momentum indicator highlights potentia...,2019-08,2019-08-15,2019-32
2,Paxful’s peer-to-peer Bitcoin exchange got a b...,BTC|Exchange|Trading|Fiat|Market,2019-08-15 21:55:41,2813832,1565906141,decrypt,Cryptocurrencies|bitcoin|cryptocurrency|Market...,Bitcoin peer-to-peer exchange Paxful sees 70 p...,bitcoin peertopeer exchange paxful sees perce...,2019-08,2019-08-15,2019-32
3,"Move over, John McAfee, there&#8217;s a new cr...",BTC|Blockchain|Technology|Regulation,2019-08-15 21:21:14,2812410,1565904074,cryptobriefing,News|Bitcoin|Government|Politics|USA,Crypto Candidate Andrew Yang Talks Up Bitcoin ...,crypto candidate andrew yang talks up bitcoin ...,2019-08,2019-08-15,2019-32
4,TRON announced the latest release of its side ...,ETH|Blockchain|BTC|Market,2019-08-15 21:19:06,2812411,1565903946,cryptoslate,Analysis|EOS|Ethereum|Tron,Is TRON’s Sun Network a marketing gimmick? Com...,is trons sun network a marketing gimmick compa...,2019-08,2019-08-15,2019-32


# 2. Preprocessing

In [15]:
#appending additional stopwords for better topic modeling
stopwords = nltk.corpus.stopwords.words('english')
crypto_stop_words = ['bitcoin', 'bitcoins', 'cryptocurrency', 'crypto', 'cryptocurrencies','first', 'new', 'post', 'pron','blockchain',
                     'cryptocurrencie', 'price', 'appear','exchange','analysis','market','ethereum', 'say','south','one']
stopwords.extend(crypto_stop_words)

In [16]:
#Simple function that lemmatizes a description
scy = spacy.load("en_core_web_sm")
def desc_prepro(desc):
    doc = scy(desc)
    words = [token.lemma_ for token in doc]
    return ' '.join(words)



In [17]:
#Lemmmatizing Process -- Takes about 10 Minutes
#df_analysis['text'] = df_analysis['text'].map(desc_prepro)
#df_analysis.to_csv('topicmodel.csv')

In [18]:
df_analysis.head()

,body,categories,date_time,id,published_on,source,tags,title,text,year_month,year_month_day,date_time_week
0,Bitcoin has been trading for a premium of betw...,BTC|Market|Trading,2019-08-15 22:00:47,2813980,1565906447,newsbtc,Bitcoin|Opinion|argentina|bitcoin|safe haven,Argentine Bitcoin Premium Shows Relativity of ...,argentine bitcoin premium shows relativity of ...,2019-08,2019-08-15,2019-32
1,Bitcoin’s market valuation has already made so...,BTC|Trading|Market,2019-08-15 22:00:21,2813979,1565906421,bitcoinist,Bitcoin|Bitcoin Price|News|News teaser|bitcoin...,Brandt’s Momentum Indicator Highlights Potenti...,brandts momentum indicator highlights potentia...,2019-08,2019-08-15,2019-32
2,Paxful’s peer-to-peer Bitcoin exchange got a b...,BTC|Exchange|Trading|Fiat|Market,2019-08-15 21:55:41,2813832,1565906141,decrypt,Cryptocurrencies|bitcoin|cryptocurrency|Market...,Bitcoin peer-to-peer exchange Paxful sees 70 p...,bitcoin peertopeer exchange paxful sees perce...,2019-08,2019-08-15,2019-32
3,"Move over, John McAfee, there&#8217;s a new cr...",BTC|Blockchain|Technology|Regulation,2019-08-15 21:21:14,2812410,1565904074,cryptobriefing,News|Bitcoin|Government|Politics|USA,Crypto Candidate Andrew Yang Talks Up Bitcoin ...,crypto candidate andrew yang talks up bitcoin ...,2019-08,2019-08-15,2019-32
4,TRON announced the latest release of its side ...,ETH|Blockchain|BTC|Market,2019-08-15 21:19:06,2812411,1565903946,cryptoslate,Analysis|EOS|Ethereum|Tron,Is TRON’s Sun Network a marketing gimmick? Com...,is trons sun network a marketing gimmick compa...,2019-08,2019-08-15,2019-32


In [114]:
vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum read occurences of a word 
                             stop_words= stopwords,             # remove stop words, using a custom list 
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=1000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(df_analysis.text)

In [115]:
#print(X)
df_vectors = pd.DataFrame(vectorizer.idf_, index=vectorizer.get_feature_names(), columns = ['Weight'])
 
# sort ascending

df_word_weights = df_vectors.sort_values(by = 'Weight').reset_index()
df_word_weights

,index,Weight
0,news,2.519669
1,world,3.224442
2,make,3.255424
3,company,3.263004
4,use,3.282426
5,also,3.311731
6,year,3.314620
7,time,3.333947
8,btc,3.344669
9,may,3.388266


In [116]:
#check word with the most weight
data_vectorized.shape

(45473, 1000)

In [117]:
data_vectorized.shape #does this mean i have exactly 1000 words?

(45473, 1000)

In [120]:
# Build LDA Model
n_components = 5

lda_model = LatentDirichletAllocation(n_components=n_components,   # Number of topics
                                      max_iter=10,                 # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,            # Random state
                                      #batch_size=128,             # n docs in each learning iter
                                      n_jobs = -1,                 # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [121]:
pyLDAvis.enable_notebook()

panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2       31.853294 -107.438812       1        1  34.765094
0      124.294113   21.438843       2        1  24.573239
4       32.204689   16.894644       3        1  22.632567
1      117.370865  -73.694336       4        1   9.513790
3      -38.172291  -42.672703       5        1   8.515310, topic_info=    Category        Freq        Term       Total  loglift  logprob
756  Default  747.000000      ripple  747.000000  30.0000  30.0000
994  Default  612.000000         xrp  612.000000  29.0000  29.0000
565  Default  759.000000      mining  759.000000  28.0000  28.0000
518  Default  559.000000    litecoin  559.000000  27.0000  27.0000
303  Default  451.000000         eos  451.000000  26.0000  26.0000
83   Default  398.000000         ban  398.000000  25.0000  25.0000
921  Default  411.000000        tron  411.000000  24.0000  24.0000
148  Default  774.000000        cash  774.000000  23.0000  23.0000
84   Default  837.000000        bank  837.000000  22.0000  22.0000
564  Default  332.000000       miner  332.000000  21.0000  21.0000
87   Default  330.000000         bch  330.000000  20.0000  20.0000
442  Default  285.000000       india  285.000000  19.0000  19.0000
219  Default  284.000000       court  284.000000  18.0000  18.0000
166  Default  277.000000     chinese  277.000000  17.0000  17.0000
165  Default  349.000000       china  349.000000  16.0000  16.0000
777  Default  265.000000        scam  265.000000  15.0000  15.0000
108  Default  247.000000     bitmain  247.000000  14.0000  14.0000
69   Default  246.000000      attack  246.000000  13.0000  13.0000
847  Default  258.000000     stellar  258.000000  12.0000  12.0000
919  Default  416.000000       trend  416.000000  11.0000  11.0000
637  Default  245.000000      patent  245.000000  10.0000  10.0000
708  Default  350.000000       range  350.000000   9.0000   9.0000
963  Default  235.000000        wall  235.000000   8.0000   8.0000
856  Default  235.000000      street  235.000000   7.0000   7.0000
949  Default  225.000000   venezuela  225.000000   6.0000   6.0000
368  Default  220.000000       fraud  220.000000   5.0000   5.0000
163  Default  438.000000       chart  438.000000   4.0000   4.0000
508  Default  410.000000       level  410.000000   3.0000   3.0000
277  Default  350.000000        drop  350.000000   2.0000   2.0000
868  Default  482.000000     support  482.000000   1.0000   1.0000
..       ...         ...         ...         ...      ...      ...
963   Topic5  234.766126        wall  235.643305   2.4596  -4.1227
856   Topic5  234.624672      street  235.501625   2.4596  -4.1233
949   Topic5  224.848130   venezuela  225.722528   2.4594  -4.1658
135   Topic5  200.157846     buterin  201.027848   2.4590  -4.2821
959   Topic5  186.169500     vitalik  187.040122   2.4586  -4.3546
775   Topic5  187.544298     satoshi  188.421900   2.4586  -4.3472
648   Topic5  172.780408       petro  173.650774   2.4583  -4.4292
443   Topic5  171.071506      indian  171.945226   2.4582  -4.4392
778   Topic5  170.113837      scheme  170.991249   2.4582  -4.4448
388   Topic5  169.177702     goldman  170.051364   2.4582  -4.4503
854   Topic5  158.267924       story  159.167108   2.4576  -4.5170
311   Topic5  155.174062      europe  156.067553   2.4576  -4.5367
497   Topic5  147.293808     lawsuit  148.165841   2.4574  -4.5888
991   Topic5  146.283786      wright  147.152691   2.4574  -4.5957
495   Topic5  145.146731  laundering  146.027115   2.4573  -4.6035
421   Topic5  140.132341        hong  141.008132   2.4571  -4.6387
392   Topic5  132.213855         gox  133.086190   2.4567  -4.6968
221   Topic5  124.109284       craig  124.978335   2.4563  -4.7601
770   Topic5  123.751409       sachs  124.623348   2.4563  -4.7630
475   Topic5  122.227074        john  123.102134   2.4562  -4.7754
806   Topic5  122.187606        shut  123.065023   2.4561  -4.7757
